# Sensor Data Analysis for recognizing letters

Required packages for this notebook:

```
pip install matplotlib numpy ipympl ipywidgets
```

In [ ]:
import dataclasses
import pathlib
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

## Data loading

In [ ]:
@dataclasses.dataclass
class Recording:
  id: str
  gravity: np.ndarray
  quat: np.ndarray

def loadData() -> List[Recording]:
  result = []
  root = pathlib.Path("feb22")
  for dir in root.glob("M_*"):
    gravity = loadNumeric(dir / "gravity_data.txt")
    quat = loadNumeric(dir / "quat_data.txt")
    result.append(Recording(id=dir.name, gravity=gravity, quat=quat))
  return result

def loadNumeric(path: pathlib.Path) -> np.ndarray:
  rows = []
  for line in open(path, 'r'):
    if not line.strip():
      continue
    rows.append(np.array([float(v) for v in line.split()], np.float64))
  return np.stack(rows, axis=0)

recordings = loadData()

## Quaternion utilities

In [ ]:
def mulQuat(q0, q1):
  """Multiply two quaternions."""
  w0 = q0[0]
  x0 = q0[1]
  y0 = q0[2]
  z0 = q0[3]

  w1 = q1[0]
  x1 = q1[1]
  y1 = q1[2]
  z1 = q1[3]

  return np.array([
    w0 * w1 - x0 * x1 - y0 * y1 - z0 * z1,
    w0 * x1 + x0 * w1 + y0 * z1 - z0 * y1,
    w0 * y1 - x0 * z1 + y0 * w1 + z0 * x1,
    w0 * z1 + x0 * y1 - y0 * x1 + z0 * w1,
  ])

def invQuat(q):
  """Invert a quaternion rotation (quat conjugate)."""
  return np.array([q[0], -q[1], -q[2], -q[3]])

def applyRotation(q, p):
  """Returns a position vector p, rotated by a unit quaternion rotation."""
  # p' = q * p * inv(q)
  return mulQuat(q, mulQuat(np.concatenate([[0], p]), invQuat(q)))[1:]

## Turning a sequence of orientations to a 3D trajectory

In [ ]:
def get3DTipTrajectory(recording: Recording):
  """Returns an Nx3 matrix of positions based on recording.quat."""
  # Direction of the wand tip relative to quat frame.
  # Chosen by trial and error to give positive Z values and clear shapes.
  # 1, 0, 0  works reasonably well too.
  tip = np.array([0.8, -0.5, -0.4], dtype=np.float64)
  tip /= np.linalg.norm(tip)

  # Drop first 10 entries as gravity is not calibrated
  quats = recording.quat[10:]

  result = np.empty([quats.shape[0], 3], np.float64)
  for i in range(quats.shape[0]):
    result[i, :] = applyRotation(quats[i, :], tip)

  return result

def plot3DTrajectory(ax, trajectory):
  """Adds a 3D scatter plot of the trajectory to `ax`."""
  [l.remove() for l in ax.lines]
  xs = trajectory[:, 0]
  ys = trajectory[:, 1]
  zs = trajectory[:, 2]
  ax.scatter(xs, ys, zs, marker='o')
  ax.scatter([0], [0], [0], marker='x')
  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_zlabel('Z')
  ax.set_xbound(-1, 1)
  ax.set_ybound(-1, 1)
  ax.set_zbound(-1, 1)

traj_fig = plt.figure('All Trajectories', clear=True)
traj_fig.clear()
traj_ax = traj_fig.add_subplot(projection='3d')

for recording in recordings:
  plot3DTrajectory(traj_ax, get3DTipTrajectory(recording))


The previous cell shows that largely, we have decent trajectories, aligned with positive Z being upwards.

Now we want to project each trajectory into a small 2D image. The idea is to convert the trajectory back to Euler angles, find the smallest yaw and pitch ranges that encompass the whole trajectory, and treat those values as X and Y in the image space.

In [ ]:
def yawAngles(trajectory):
  """Converts position on unit circle to yaw angles (longitude).
  
  Returns angles in [-pi, pi]."""
  # find the angle on the x-y plane.
  return np.arctan2(trajectory[:, 0], trajectory[:, 1])

def pitchAngles(trajectory):
  """Converts positions on unit circle to pitch angles (latitude).
  
  Returns angles in [-pi/2, pi/2]."""
  return np.arcsin(trajectory[:, 2])

def wrappingAngleRange(angles) -> Tuple[float, float]:
  """Returns the smallest angle range that encompasses all angles.
  
  Args:
    angles: array of radian angles in [-pi, pi].
  
  Returns:
    a tuple of two angles. If the second is smaller than the first, then the
    angle range contains pi.
  """
  # Algorithm: Find the largest gap between adjacent angles (handling the wrap-around
  # at pi), then return 2*pi minus that angle.
  angles = np.sort(angles)
  angles_diff = angles - np.roll(angles, 1)
  angles_diff[0] = 2*np.pi + angles_diff[0]
  first_angle = np.argmax(angles_diff)
  last_angle = (first_angle - 1) % angles.size
  return (angles[first_angle], angles[last_angle])

for i in range(len(recordings)):
  angle_range = wrappingAngleRange(yawAngles(get3DTipTrajectory(recordings[i])))
  print(i, "[%0.1f, %0.1f]" % (angle_range[0] * 180 / np.pi, angle_range[1] * 180 / np.pi))


In [23]:
def normaliseTrajectory(trajectory):
  """Returns trajectory points as yaw and pitch angles, normalised to start at 0."""
  yaw = yawAngles(trajectory)
  pitch = pitchAngles(trajectory)
  yaw_range = wrappingAngleRange(yaw)
  # note: wrappingAngleRange would work for pitch, but we don't need it, since the
  # angles are in [-pi/2, pi/2] and don't wrap around
  pitch_range = (np.min(pitch), np.max(pitch))
  yaw -= yaw_range[0]
  yaw[yaw < 0] += 2 * np.pi
  pitch = pitch - pitch_range[0]
  pitch[pitch < 0] += 2 * np.pi

  return fitInBox(yaw, pitch)

def fitInBox(x: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  """Rescales a set of x-y coordinates to fit in a 1x1 box."""
  min_x = np.min(x)
  x_range = np.max(x) - min_x
  min_y = np.min(y)
  y_range = np.max(y) - min_y
  scale = 1.0 / max(x_range, y_range)
  new_x = 0.5 + (x - min_x - x_range / 2) * scale
  new_y = 0.5 + (y - min_y - y_range / 2) * scale
  
  # print("new_x[0:15]",new_x[0:15])
  # print("new_y[0:15]",new_y[0:15])
  return new_x, new_y

def plotNormalized(ax, trajectory, name):
  yaw, pitch = normaliseTrajectory(trajectory)
  ax.scatter(x=yaw, y=pitch, alpha=0.2, label=name)

normalized_fig = plt.figure("norm")
normalized_fig.clear()
normalized_ax = normalized_fig.add_subplot()
for i, recording in enumerate(recordings):
  plotNormalized(normalized_ax, get3DTipTrajectory(recording), str(i))
  break
normalized_ax.axis('equal')
normalized_ax.legend()

min_x 0.0
x_range 0.4174163668901669
min_y 0.0
y_range 0.6117192027830682
scale 1.6347369764598128
new_x [0.15881702 0.1601148  0.16182179 0.1637163  0.16538318 0.16758462
 0.17032914 0.17371495 0.17791385 0.18229903 0.18660505 0.19038098
 0.19356424 0.19619203 0.19853121 0.20074081 0.20282309 0.20512278
 0.20736861 0.20864351 0.20941296 0.20920477 0.20869863 0.2085482
 0.20938905 0.21121601 0.21365218 0.21702114 0.22101274 0.22550005
 0.23064313 0.23595619 0.24168937 0.24721776 0.25230121 0.25680631
 0.26119687 0.26615727 0.27190387 0.27894618 0.28511933 0.29102779
 0.29578256 0.3000662  0.3073275  0.3073275  0.31135201 0.31481498
 0.31715354 0.31836087 0.31799722 0.3169344  0.31551932 0.31407566
 0.3130748  0.31262159 0.31353923 0.31457953 0.31585949 0.31660567
 0.31766668 0.31870699 0.3200439  0.32210507 0.3242871  0.32663294
 0.32924705 0.33256294 0.33702547 0.34287831 0.35115495 0.36073365
 0.37145445 0.38320494 0.39562178 0.40912287 0.42329325 0.43789166
 0.45295637 0.46888646 0.

## Appendix: Data validation - understanding the gravity vector

Validate that gravity vectors are about 9.81 m/s^2 in magnitude.

In [ ]:
def gravityNormsHistogram(recordings):
  gravity_norms = np.concatenate(
    [np.linalg.norm(recording.gravity, axis=1) for recording in recordings])
  fig = plt.figure("gravity_norm")
  ax = fig.add_subplot()
  ax.hist(x=gravity_norms, bins=30)
  ax.set_xlabel("Gravity vector magnitude")
  ax.set_ylabel("Datapoint count")

gravityNormsHistogram(recordings)

John says each gravity list is missing the first 10 elements to align with quaternions. Verify:

In [ ]:
def validateDimensions(recordings):
  for recording in recordings:
    g_dim = recording.gravity.shape[0]
    quat_dim = recording.quat.shape[0]
    if g_dim + 10 != quat_dim:
      raise ValueError(f"Recording {recording.id} has {g_dim} gravity readings and {quat_dim} quat entries")
validateDimensions(recordings)

Try averaging the gravity vectors. If they always point the same way between trajectories, we'd would expect the mean to have magnitude close to 9.81 m/s^2 (since we won't have destructive interference).

In [ ]:
def meanGravity(recordings, firstn=None):
  if firstn:
    return np.mean(np.concatenate(
        [recording.gravity[:firstn, :] for recording in recordings], axis=0), axis=0)
  return np.mean(np.concatenate(
    [recording.gravity for recording in recordings], axis=0), axis=0)

print("Overall mean:", meanGravity(recordings), np.linalg.norm(meanGravity(recordings)))
print("First step mean:", meanGravity(recordings, 1), np.linalg.norm(meanGravity(recordings, 1)))

Results show that the gravity vectors don't tend to be consistent between episodes. This could be due to differences in how the sensor is mounted, differences in how the wand is held, or a calibration issue.

Next: Find a relation between the quaternions and the gravity vector.

In [ ]:
def rotatedGravity(recording: Recording, transpose: bool = False):
  quats = recording.quat[10:]
  result = np.empty([quats.shape[0], 3], np.float64)
  for i in range(quats.shape[0]):
    result[i, :] = applyRotation(quats[i, :], recording.gravity[i, :])

  return result

def meanRotatedGravity(recordings: List[Recording]) -> np.ndarray:
  return np.mean(np.concatenate([rotatedGravity(r) for r in recordings], axis=0), axis=0)

# Prints a vector like
# [[ 0.23779003]
#  [-0.37204996]
#  [ 9.7412319 ]]
# which suggests that:
# 1. This is the correct transformation (close to 9.81 in the same direction)
# 2. The base orientation has Z going downwards
print(meanRotatedGravity(recordings),
      np.linalg.norm(meanRotatedGravity(recordings)))

So now we have a way of taking a vector in the wand frame (e.g. gravity)
and getting a result in the world frame, but the world frame has positive Z going downwards.
We should be able to pick a point along the wand line (e.g. [1, 0, 0]), transform it with the
quaternions, and get its trajectory in the world frame.